In [3]:
import os
wd= 'C:/Users/iamph/Documents/GitHub/hbv-globalinvcase'#set own working directory
os.chdir(wd)

import atomica as at
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

In [4]:
region = 'AMR'
F = at.ProjectFramework('frameworks/hbv_v14_gamma_mav.xlsx')
D = at.ProjectData.from_spreadsheet(f"applications/region_{region.lower()}/{region}_db_mav.xlsx", framework=F)
P= at.Project(framework=F, databook=D, sim_dt=0.25, sim_start=1990, sim_end=2099, do_run=False)
cal = P.make_parset()

C:\Users\iamph\Miniconda3\lib\site-packages\atomica\framework.py:914: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for to_comp, par_names in from_row.iteritems():


In [5]:
res = P.run_sim(parset=cal, result_name = 'Base Calibration')

Elapsed time for running "default": 4.82s


### Extract Key Datapoints

In [7]:
def metric(data, est, mt):
    """
    LAST EDIT: 10/08/21
    by: Phillip Luong

    :param data: An array or list of data points to calibrate to
    :param est: An array or list of the estimated points
    :param mt: 'mae', 'mse', 'rmse', 'mape'
    :return: Numercal value for the error
    """
    if mt == "mae":
        return mean_absolute_error(data, est, sample_weight=data)
    if mt == "mse":
        return mean_squared_error(data, est, sample_weight=data)
    if mt == "rmse":
        return mean_squared_error(data, est, sample_weight=data, squared=False)
    if mt == "mape":
        return mean_absolute_percentage_error(data, est, sample_weight=data)

In [15]:
edf = pd.DataFrame()
p_years = [2015,2019]
years = [2010, 2015, 2019]

In [8]:
len(D.pops)

10

In [12]:
data = [P.data.tdve["cl_cir"].ts[x].interpolate(2010) for x in range(10)]
data = [i[0] for i in data]

In [13]:
data

[1.0, 0.0, 1.0, 0.0, 2188.0, 388.0, 2902.0, 719.0, 1035.0, 516.0]

In [20]:
res.get_variable("alive")[0]

Characteristic "alive" ('0-4M', 'alive')

In [22]:
[x.vals[ * 4] for x in res.get_variable("alive")[0]]

AttributeError: 'numpy.float64' object has no attribute 'vals'

In [26]:
data = []
est = []
for yr in years[1:]:
    tmp = [P.data.tdve["alive"].ts[x].interpolate(2000 + yr) for x in range(10)]  # data
    data += [i[0] for i in tmp]
    est += [x.vals[(yr-2000) * 4] for x in res.get_variable("alive")]  # estimate

In [25]:
len(est)

20

In [27]:
len(data)

20

In [28]:
metric(data, est, 'mape')

0.5064597681882885

In [85]:
regions = ['AFR', 'AMR', 'EMR', 'EUR', 'SEAR', 'WPR']

In [40]:
pars = ['chb_pop', 'cl_acu', 'cl_cir', 'cl_hcc']

In [76]:
ct

'AMR'

In [86]:
# regions = ['AFR']
edf = pd.DataFrame()
years = [2010, 2015, 2019]
F = at.ProjectFramework('frameworks/hbv_v14_gamma_mav.xlsx')
for ct in regions:
    print(f"Current region: {ct}")
    D = at.ProjectData.from_spreadsheet(f"applications/region_{ct.lower()}/{ct}_db_mav.xlsx", framework=F)
    P= at.Project(framework=F, databook=D, sim_dt=0.25, sim_start=1990, sim_end=2099, do_run=False)
    cal = P.make_parset()
    cal.load_calibration(f"applications/region_{ct.lower()}/{ct}_calib.xlsx")
    res = P.run_sim(parset=cal)
    for par in pars:
        data = []
        est = []
        if par == 'chb_pop':
            tmp_yr = years[1:]
        else:
            tmp_yr = years
        for yr in tmp_yr:
            tmp = [P.data.tdve[par].ts[x].interpolate(2000 + yr) for x in range(10)]  # data
            data = [i[0] for i in tmp]
            est = [x.vals[(yr-2000) * 4] for x in res.get_variable(par)]  # estimate

            edf.loc[f"{ct}", par] = metric(data, est, "mape")
    
edf = edf.apply(pd.to_numeric).round(4)

C:\Users\iamph\Miniconda3\lib\site-packages\atomica\framework.py:914: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for to_comp, par_names in from_row.iteritems():


Current region: AFR
Elapsed time for running "default": 3.40s
Current region: AMR
Elapsed time for running "default": 3.46s
Current region: EMR
Elapsed time for running "default": 4.08s
Current region: EUR
Elapsed time for running "default": 3.81s
Current region: SEAR
Elapsed time for running "default": 3.64s
Current region: WPR
Elapsed time for running "default": 4.10s


In [87]:
edf

,chb_pop,cl_acu,cl_cir,cl_hcc
AFR,0.2517,0.1349,0.0847,0.1731
AMR,0.2972,0.2728,0.1721,0.2041
EMR,0.3038,0.5231,0.2043,0.2167
EUR,0.1721,0.3919,0.1634,0.0910
SEAR,0.2581,0.1978,0.1101,0.1960
WPR,0.3053,0.4285,0.1827,0.1599


In [81]:
regions = ['AMR']
edf = pd.DataFrame()
years = [2010, 2015, 2019]
F = at.ProjectFramework('frameworks/hbv_v14_gamma_mav.xlsx')
for ct in regions:
    print(f"Current region: {ct}")
    D = at.ProjectData.from_spreadsheet(f"applications/region_{ct.lower()}/{ct}_db_mav.xlsx", framework=F)
    P = at.Project(framework=F, databook=D, sim_dt=0.25, sim_start=1990, sim_end=2099, do_run=False)
    cal = P.make_parset()
    cal.load_calibration(f"applications/region_{ct.lower()}/{ct}_calib.xlsx")
    res = P.run_sim(parset=cal)
    for par in pars:
        data = []
        est = []
        if par == 'chb_pop':
            tmp_yr = years[1:]
        else:
            tmp_yr = years
        for yr in tmp_yr:
            data = [P.data.tdve[par].ts[x].interpolate(2000 + yr) for x in range(10)]  # data
            data = [i[0] for i in data]
            est = [x.vals[(yr-2000) * 4] for x in res.get_variable(par)]  # estimate

            edf.loc[f"{ct}", par] = metric(data, est, "mape")
    
edf = edf.apply(pd.to_numeric).round(4)

C:\Users\iamph\Miniconda3\lib\site-packages\atomica\framework.py:914: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for to_comp, par_names in from_row.iteritems():


Current region: AMR
Elapsed time for running "default": 3.66s


In [82]:
edf

,chb_pop,cl_acu,cl_cir,cl_hcc
AMR,0.2972,0.2728,0.1721,0.2041


In [83]:
data

[0.0, 0.0, 8.0, 3.0, 852.0, 510.0, 3508.0, 1546.0, 2149.0, 1301.0]

In [84]:
est

[0.0,
 0.0,
 12.218205251117228,
 4.0627073109801355,
 976.627020355523,
 504.05955771164736,
 2740.5966395963433,
 1184.0580276482715,
 1644.046787548729,
 1054.8450249518794]

### Sum over all populations

### Create correct error functions

### Colour-code error tables

In [88]:
def select_col_blk(x):
    """
    GOAL: Generalise this for incidences as well

    """
    c1 = "background-color: grey"
    c2 = ""
    mask_cols = [i for i in x.columns if "2040" in i]
    mae_col = [i for i in mask_cols if "mae" in i][0]
    mask_cols.remove(mae_col)
    mape_col = mask_cols[0]

    # compare columns
    mask = (x[mape_col] > 1) & (x[mae_col] < 10)
    # mask = x['par_2040_death'] > 0.6
    # DataFrame with same index and columns names as original filled empty strings
    df1 = pd.DataFrame(c2, index=x.index, columns=x.columns)
    # modify values of df1 column by boolean mask
    df1.loc[mask, mape_col] = c1
    return df1


def select_col_red(x):
    """
    GOAL: Generalise this for incidences as well

    """
    c1 = "background-color: lightcoral"
    c2 = ""
    mask_cols = [i for i in x.columns if "2040" in i]
    mae_col = [i for i in mask_cols if "mae" in i][0]
    mask_cols.remove(mae_col)
    mape_col = mask_cols[0]

    # compare columns
    mask = (x[mape_col] > 0.6) & (x[mape_col] < 1) & (x[mae_col] < 10)
    # mask = x['par_2040_death'] > 0.6
    # DataFrame with same index and columns names as original filled empty strings
    df1 = pd.DataFrame(c2, index=x.index, columns=x.columns)
    # modify values of df1 column by boolean mask
    df1.loc[mask, mape_col] = c1
    return df1

In [89]:
def highlight_results(s, lb, ub, props=""):
    return np.where((lb <= s) & (s < ub), props, "")

In [97]:
def determine_cal_quality(df):
    """
    Needs highlight_results, helps determine how good the calibrations are after generate_output_df
    """

    df = df.style.apply(highlight_results, lb=1, ub=1000, props="color:white;background-color:black",  axis=0).apply(highlight_results, lb=0.6, ub=1, props="color:white;background-color:red",axis=0).apply(highlight_results, lb=0.4, ub=0.6, props="color:black;background-color:gold",  axis=0).apply(highlight_results, lb=0, ub=0.4, props="color:black;background-color:mediumseagreen",  axis=0)
    return df

In [98]:
determine_cal_quality(edf)

,chb_pop,cl_acu,cl_cir,cl_hcc
AFR,0.251700,0.134900,0.084700,0.173100
AMR,0.297200,0.272800,0.172100,0.204100
EMR,0.303800,0.523100,0.204300,0.216700
EUR,0.172100,0.391900,0.163400,0.091000
SEAR,0.258100,0.197800,0.110100,0.196000
WPR,0.305300,0.428500,0.182700,0.159900
